Chain of thought prompting

In [15]:
!pip install transformers torch

In [2]:
from transformers import pipeline

In [3]:
generator = pipeline(
    "text-generation",
    model="google/flan-t5-base"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausa

In [4]:
def chain_of_thought_prompt(user_question):
    prompt = f"""
Solve the following problem step by step and explain your reasoning clearly.

Question:
{user_question}

Answer:
"""
    return prompt

In [5]:
def run_cot(question):
    prompt = chain_of_thought_prompt(question)

    response = generator(
        prompt,
        max_length=200,
        do_sample=False
    )

    print("USER QUESTION:")
    print(question)
    print("\nCHAIN OF THOUGHT RESPONSE:\n")
    print(response[0]["generated_text"])

In [6]:
question = "A student studies 4 hours per day for 6 days. How many hours does the student study in total?"

run_cot(question)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'max_length', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


USER QUESTION:
A student studies 4 hours per day for 6 days. How many hours does the student study in total?

CHAIN OF THOUGHT RESPONSE:


Solve the following problem step by step and explain your reasoning clearly.

Question:
A student studies 4 hours per day for 6 days. How many hours does the student study in total?

Answer:
 in total so he studies 96 + 96 = 108 hours. The answer: 108.


Tree of thought prompting

In [7]:
!pip install transformers torch

In [8]:
#load model from hugging face
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_text(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generate_text("Solve step by step: What is 12 * 8?"))

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


3 square bases n can match on one platform


In [9]:
#helper function: generate text
def generate_text(prompt, max_new_tokens=80):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
#Step 1: Generate multiple thoughts (branches)
def generate_thoughts(question, k=3):
    thoughts = []
    for i in range(k):
        prompt = f"""
Question: {question}
Think step by step and suggest one possible reasoning path.
Thought:
"""
        thought = generate_text(prompt)
        thoughts.append(thought)
    return thoughts

In [11]:
# Step 2: Expand each thought (Tree growth 🌳)
def expand_thought(thought):
    prompt = f"""
Here is a reasoning step:
{thought}

Continue this reasoning logically:
"""
    return generate_text(prompt)

In [12]:
#Step 3: Score thoughts (simple heuristic)
#we score by length + presence of numbers / logic words.
def score_thought(thought):
    score = len(thought)
    logic_keywords = ["because", "therefore", "so", "hence", "thus"]

    for word in logic_keywords:
        if word in thought.lower():
            score += 20
    return score

In [13]:
#Step 4: Tree of Thought solver
def tree_of_thought_solver(question, branches=3):
    print("🌱 Generating initial thoughts...\n")
    thoughts = generate_thoughts(question, branches)

    expanded = []
    for t in thoughts:
        e = expand_thought(t)
        expanded.append(e)

    scored = [(score_thought(t), t) for t in expanded]
    scored.sort(reverse=True, key=lambda x: x[0])

    best_thought = scored[0][1]

    final_prompt = f"""
Based on the reasoning below, give the final answer clearly.

Reasoning:
{best_thought}

Final Answer:
"""

    final_answer = generate_text(final_prompt)
    return final_answer

In [14]:
question = "If a train travels 60 km in 1 hour and then 30 km in 30 minutes, what is its average speed?"

answer = tree_of_thought_solver(question)
print("✅ Final Answer:\n", answer)

🌱 Generating initial thoughts...

✅ Final Answer:
 It starts around 1600 and it last after 400 seconds from working on total of 7070 units but with work for the first part there. If 600:800/2260/7540 means the operating price that makes 400/1000 and 4/10:40 3 is less pricey (not used all in favor to cheap ratio when looking from ratios you probably like but you had not) to do 300


In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate(prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [20]:
problem = "A farmer has 17 sheep and all but 9 die. How many are left?"

initial_prompt = f"""
Solve the following problem using different reasoning approaches.

Problem: {problem}

Approach 1:
"""

nodes = [generate(initial_prompt) for _ in range(3)]

print("Initial Thought Nodes:\n")
for i, node in enumerate(nodes):
    print(f"Node {i+1}:\n{node}\n")


Initial Thought Nodes:

Node 1:
57 days ago 16 goat will finish 76 goats who have done 1 dead disease; 4 is healthy and they know what this recipe could possibly sound

Node 2:
11 from one harvest would kill 10 animals with 0.5 per farm day which was one. If 8 werenaties and they all are removed at 11, all 17, should fall because 30 people fought their hard hard for 16 to 11 left since 11 years was 8 from an end all but 29 in 12 from 2 killed from 19 the farmer then 9 on 92 killing total after 22 that lost its rights 24 of 25 = 42 years age difference for 10 left after leaving it to 28: 51 animals

Node 3:
4 in percent leaves are 37 dead +1 8 is not included of 11 days (43 min * 11 min max per minute); 25 for unrelated crops 2 will find 1 + 30 on any 2 for remaining 2 for those making ends set 4 should use 4 of 18 the days 7 not listed for these days to end; 7 don



In [21]:
expanded_nodes = []

for node in nodes:
    expand_prompt = f"""
Continue and refine this reasoning carefully:

{node}

Give a clear final answer.
"""
    expanded_nodes.append(generate(expand_prompt))

print("Expanded Nodes:\n")
for i, node in enumerate(expanded_nodes):
    print(f"Expanded {i+1}:\n{node}\n")


Expanded Nodes:

Expanded 1:
12 and 16 live the healthy yeast who produce all its vegetables at your source material one, six sheep shall grow every 101 and 16 one can spread any digestive matter like fish seed in addition just five of 3 which causes blood problems 2 has the same body skin condition, five cow and several species which means an digestive component such more to come one year earlier 2 1 so in our country 2 in 15 + we should bree back 4th out there 7 years younger with healthy egg is 

Expanded 2:
Farmers give 17 for each less per end equal totaling around half 16, with half less dead trees over 25 because 20 died by 25 and 10 would take 12 and 8; 25 should die, 5 won to 20; 20 losses: 21 will equal 8 lost = 72: 46 of every 30 = 25 humans except one so there we will only go away = 45 that no end die to, 45 the remainder which has survived 10 being left one after this will save 60 each being saved on 17 

Expanded 3:
21 from 9 were NOT mentioned as long that this equal 2. 

In [22]:
merge_prompt = f"""
We have multiple reasoning attempts.

Solution 1:
{expanded_nodes[0]}

Solution 2:
{expanded_nodes[1]}

Solution 3:
{expanded_nodes[2]}

Combine the correct reasoning and give ONLY the final correct answer.
"""

final_answer = generate(merge_prompt)

print("Final Merged Answer:\n")
print(final_answer)


Final Merged Answer:

Solutions 20 is less that. 5 and 21 we got more the end outcomes 20 with one in which an insect or insects the third can come together to be tested. we saw 30 which can no., even it seems in reality 20 should stay
